# 1_loading.ipynb

- Goal: Merge twitter data from twitter API
- from json format to dataframe and csv


In [1]:
# import packages
import re
import pandas as pd
import pandas as pd
import re 
import gensim
from gensim.parsing.preprocessing import STOPWORDS, strip_tags, strip_numeric, strip_punctuation, strip_multiple_whitespaces, remove_stopwords, strip_short, stem_text
import pickle
import en_core_web_sm
import nltk
import os
from nltk.tokenize import word_tokenize
import emoji

D:\deniz\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
# filter files only for our big data project, any language
all_files = os.listdir(path='twitterdata')
r = re.compile(".*_(movie)")
files = list(filter(r.match, all_files)) 
len(files)

14

In [18]:
#make one dataframe out of all these files
dfs = []

files_paths = ['twitterdata/' + file for file in files]

for x in files_paths:
    data = pd.read_json(x)
    dfs.append(data)

df = pd.concat(dfs)

In [19]:
# overview of the df
df.shape

(10965, 13)

In [20]:
df.columns

Index(['user_id', 'user_statuses', 'name', 'tweet', 'retweets', 'location',
       'created', 'followers', 'is_user_verified', 'favorite_count',
       'in_reply_to_status_id', 'keyword', 'language'],
      dtype='object')

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10965 entries, 0 to 99
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   user_id                10965 non-null  int64  
 1   user_statuses          10965 non-null  int64  
 2   name                   10965 non-null  object 
 3   tweet                  10965 non-null  object 
 4   retweets               10965 non-null  int64  
 5   location               10965 non-null  object 
 6   created                10965 non-null  object 
 7   followers              10965 non-null  int64  
 8   is_user_verified       10965 non-null  bool   
 9   favorite_count         10965 non-null  int64  
 10  in_reply_to_status_id  4901 non-null   float64
 11  keyword                10965 non-null  object 
 12  language               10965 non-null  object 
dtypes: bool(1), float64(1), int64(5), object(6)
memory usage: 1.1+ MB


In [22]:
df.head()

,user_id,user_statuses,name,tweet,retweets,location,created,followers,is_user_verified,favorite_count,in_reply_to_status_id,keyword,language
0,1308491138077274113,9281,GoaFetz,@DFliesstext ... die schon vor weit über einem...,0,Berlin,11-Jun-2021,346,False,0,1.403253e+18,movie,de
1,18772279,5917,outnow,Seit dieser Woche läuft «Demon Slayer the Movi...,0,Zürich,11-Jun-2021,3321,False,0,NaN,movie,de
2,883643872747835392,253,subhajit_mshra,#6\n\nMehndi Hai Rachnewali\n\nMovie - Zubeida...,0,"Mumbai, India",11-Jun-2021,181,False,0,1.403252e+18,movie,de
3,4273473912,15425,ynadevs,Movie marathon,0,,11-Jun-2021,619,False,1,NaN,movie,de
4,23049076,19246,Nimbert,Anderer Ansatz als die bisherigen 12 #XMen Fil...,0,"Europe, Germany, Hannover",11-Jun-2021,238,False,0,NaN,movie,de


In [23]:
df.drop(['user_statuses', 'name', 
        'followers', 'is_user_verified', 'favorite_count',
       'in_reply_to_status_id'], axis=1, inplace=True)


In [24]:
df.head()

,user_id,tweet,retweets,location,created,keyword,language
0,1308491138077274113,@DFliesstext ... die schon vor weit über einem...,0,Berlin,11-Jun-2021,movie,de
1,18772279,Seit dieser Woche läuft «Demon Slayer the Movi...,0,Zürich,11-Jun-2021,movie,de
2,883643872747835392,#6\n\nMehndi Hai Rachnewali\n\nMovie - Zubeida...,0,"Mumbai, India",11-Jun-2021,movie,de
3,4273473912,Movie marathon,0,,11-Jun-2021,movie,de
4,23049076,Anderer Ansatz als die bisherigen 12 #XMen Fil...,0,"Europe, Germany, Hannover",11-Jun-2021,movie,de


In [25]:
def clean_complete(tweet):
    """
    tweet: pandas series
    prepares tweets complete cleaning for further lemmatization and dering embeddings
    """
    pat = r"(\\n)|(@\w*)|((www\.[^\s]+)|(https?://[^\s]+))"
    tweet = tweet.str.replace(pat, '')

    #remove repeated charachters
    
    #replace emoticons with words
    #SMILEYS = {":-(":"sad", ":‑)":"smiley", ":-P":"playfullness", ":-/":'confused'}

    tweet = tweet.str.replace(r':-\)', ' smile')
    tweet = tweet.str.replace(r':-\(', ' sad')
    tweet = tweet.str.replace(r':-\/', ' confused')
    tweet = tweet.str.replace(r':-P', ' playfullness')

    #delete \xa
    tweet = tweet.str.replace('\xa0', '')

    tweet = tweet.str.replace('&amp', '')
    tweet = tweet.str.replace('\n', '')
    tweet = tweet.str.replace('"', '')
    tweet = tweet.str.replace('ℐℓ٥ﻻﻉ√٥υ', '')
    
    #to lower case
    tweet = tweet.str.lower()

    #covert hashtags to the normal text
    tweet = tweet.str.replace(r'#([^\s]+)', r'\1')

    #delete numbers
    tweet = [strip_numeric(c) for c in tweet]

    #replacing emojies with descriptions '❤️-> red heart'
    #tweet = [demoji.replace_with_desc(c, ' ') for c in tweet]

    #replacing emojies with ''
    #tweet = [demoji.replace(c, ' ') for c in tweet]

    #delete punctuation
    tweet = [strip_punctuation(c) for c in tweet]

    #remove stop words
    tweet = [remove_stopwords(c) for c in tweet]

    #remove short words
    tweet = [strip_short(c) for c in tweet]

    #remove mult whitespaces
    tweet = [strip_multiple_whitespaces(c) for c in tweet]
    return tweet


In [26]:
df['tweet'] = clean_complete(df.tweet)

In [29]:
df.head()

,user_id,tweet,retweets,location,created,keyword,language
0,1308491138077274113,die schon vor weit über einem jahr als solche ...,0,Berlin,11-Jun-2021,movie,de
1,18772279,seit dieser woche läuft «demon slayer movie mu...,0,Zürich,11-Jun-2021,movie,de
2,883643872747835392,mehndi hai rachnewalimovie zubeida music rahman,0,"Mumbai, India",11-Jun-2021,movie,de
3,4273473912,movie marathon,0,,11-Jun-2021,movie,de
4,23049076,anderer ansatz als die bisherigen xmen filme a...,0,"Europe, Germany, Hannover",11-Jun-2021,movie,de


In [27]:
#df.reset_index

In [28]:
df.to_csv('twitter.csv', sep=',', header=False)

In [25]:
# df.reset_index(inplace=True)
# df.to_json('data.json')

In [10]:
# check value counts for all predefined kexwords ind 0_twitter_api.pynb
df.keyword.value_counts()

movie        10956
movie2021        9
Name: keyword, dtype: int64

In [ ]:
d